In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
            .appName('test-session') \
            .master('local[*]') \
            .getOrCreate()
print(spark)

In [ ]:
'''
parquet로 저장한 tier1 데이터의 스키마중에 timestamp값 명시해야함
이전 dmc데이터 처리를 그대로 따라하긴 했는데, 생각해보니 실시간으로 쌓이는 데이터를 시간단위로 쪼갠것이
서드파티 데이터처리에 맞춘것인지 아님 데이터 단위를 직관적으로 최소화해 쪼갠것인지 구분을 못하겠음..
로우데이터->표준화->필요에따른 그룹별 데이터

표준화시킨 데이터를 s3에 올려서 emr spark로 돌려보면 좋을것같다
700mb * 30일 * 12달 = 300GB~400GB  -> 1달 23000원 정도? 
emr spark 사용하는 비용은.... 

로컬에서 테스트 돌려보는건 좋은데 클러스터 상황에서 어떤 퍼포먼스가 나오는지 알수가 없네..
일단... 파일전체를 핸들링하는건지 구분하기 어려운데, timestamp를 제대로 처리하지 못하면 곤란함... 
* 일단위로 재처리해야할것같음

'''

In [7]:
# schema 확인
data_path = '/home/data/data-warehouse/tier1/event/2020-12-01/00/*'
df = spark.read.format('parquet') \
    .option('header',True) \
    .option('compression', 'gzip') \
    .load(data_path)    
df.printSchema()
df.show(1,False)

root
 |-- tracking: string (nullable = true)
 |-- trackingId: string (nullable = true)
 |-- trackingEventCode: string (nullable = true)
 |-- cid: string (nullable = true)
 |-- osTypeCode: string (nullable = true)
 |-- logTimeStamp: timestamp (nullable = true)
 |-- eventTimeStamp: timestamp (nullable = true)
 |-- campaign: string (nullable = true)
 |-- contentId: string (nullable = true)
 |-- contentName: string (nullable = true)
 |-- value: integer (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- amount: integer (nullable = true)
 |-- currency: string (nullable = true)
 |-- activityParam: string (nullable = true)
 |-- attributed: string (nullable = true)
 |-- latdAdvertisingPartnerName: string (nullable = true)

+--------+----------------------+-----------------+------------------------------------+----------+---------------------+---------------------+--------+---------+--------------------------------------------+-----+--------+------+--------+-------------------------

In [3]:
from pyspark.sql.functions import *
# 월별매출
data_path = '/home/data/data-warehouse/tier1/event/2021-01-01/*'
spark.read.format('parquet') \
        .option('header',True) \
        .option('compression', 'gzip') \
        .load(data_path)\
        .createOrReplaceTempView('t_master')


In [4]:
sql_temp = """
    select 
        date(eventTimeStamp)
    from t_master
    where 1=1
        and date(eventTimeStamp) BETWEEN date('2021-01-01') AND date('2021-01-02')        
"""

sql = """
    select        
        trackingId,
        osTypeCode,
        count(amount) as purchased_count,
        sum(value) as total_value,
        sum(amount) as total_amount,
        avg(amount) as avg_amount,
        eventTimeStamp

    from t_master
    where 1=1
        and date(eventTimeStamp) BETWEEN date('2021-01-01') AND date('2021-01-02')
    group by trackingId, osTypeCode, eventTimeStamp
    order by trackingId, osTypeCode, eventTimeStamp
"""

# processDate BETWEEN STR_TO_DATE('2021-01-01', '%Y-%m-%d') AND STR_TO_DATE('2021-02-01', '%Y-%m-%d')
# processDate BETWEEN date_format(date_add(now(), INTERVAL -90 DAY)), "%Y-%m-%d") AND date_format(date_add(now(), INTERVAL -1 DAY)), "%Y-%m-%d")
# spark.sql(sql).show(5)

        # cast(eventTimeStamp/1000 as timestamp) as eventDate,
        # from_unixtime(cast(eventTimeStamp/1000 as bigint), 'yyyy-MM-dd HH:mm:ss') as eventDate2



spark.sql(sql).show(5,False)

+----------+----------+---------------+-----------+------------+----------+--------------+
|trackingId|osTypeCode|purchased_count|total_value|total_amount|avg_amount|eventTimeStamp|
+----------+----------+---------------+-----------+------------+----------+--------------+
+----------+----------+---------------+-----------+------------+----------+--------------+



In [ ]:
data_path = '/home/data/data-warehouse/tier1/event/*'
df = spark.read.format('parquet') \
    .option('header',True) \
    .option('compression', 'gzip') \
    .load(data_path) \
    .createOrReplaceTempView('t_master')
df.where('processDate >= 2021-01-01')